# **Running Unreal engine and Airsim on Colab**

In [3]:
#@title **GPU Check**
#!nvidia-smi

from IPython.display import clear_output

FileNotFoundError: The command was not found or was not executable: /bin/bash/.

In [ ]:
#@title **Create User**
#@markdown The simulation environment will refuse to open with the current user. For this we need to add a new user who does not have sudo permissions.

#@markdown

#@markdown Username : RLTurkey

#@markdown Password : 1881 


username = "RLTurkey"
password = "1881"

print("Creating User and Setting it up")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User Created and Configured")

Creating User and Setting it up
User Created and Configured


In [ ]:
#@title **Connect to Google Drive**
#@markdown We will move the simulation to the machine in Colab via Drive and run it, you can access the simulation environment with the "cd /content/LinuxNoEditor" command.

from google.colab import drive
drive.mount('/content/drive')

!cp -R /content/drive/MyDrive/UnrealEngineRelated/LinuxZip/LinuxNoEditor /content/
!sudo chmod -R +755 /content/LinuxNoEditor
clear_output()
print("Done")

Done


In [ ]:
#@title **OpenGL setup**
#@markdown In order to run the simulation on the GPU, this step should run smoothly. Otherwise, the simulation graphics and animations run on the CPU and you will get low FPS in the simulation.
import subprocess, urllib.request, shutil, pathlib, stat, re
import apt, apt.debfile, ipywidgets

def _download(url, path):
  try:
    with urllib.request.urlopen(url) as response:
      with open(path, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)
  except:
    print("Failed to download ", url)
    raise

class _NoteProgress(apt.progress.base.InstallProgress, apt.progress.base.AcquireProgress, apt.progress.base.OpProgress):
  def __init__(self):
    apt.progress.base.InstallProgress.__init__(self)
    self._label = ipywidgets.Label()
    display(self._label)
    self._float_progress = ipywidgets.FloatProgress(min = 0.0, max = 1.0, layout = {'border':'1px solid #118800'})
    display(self._float_progress)

  def close(self):
    self._float_progress.close()
    self._label.close()

  def fetch(self, item):
    self._label.value = "fetch: " + item.shortdesc

  def pulse(self, owner):
    self._float_progress.value = self.current_items / self.total_items
    return True

  def status_change(self, pkg, percent, status):
    self._label.value = "%s: %s" % (pkg, status)
    self._float_progress.value = percent / 100.0

  def update(self, percent=None):
    self._float_progress.value = self.percent / 100.0
    self._label.value = self.op + ": " + self.subop

  def done(self, item=None):
    pass

class _MyApt:
  def __init__(self):
    self._progress = _NoteProgress()
    self._cache = apt.Cache(self._progress)

  def close(self):
    self._cache.close()
    self._cache = None
    self._progress.close()
    self._progress = None

  def update_upgrade(self):
    self._cache.update()
    self._cache.open(None)
    self._cache.upgrade()

  def commit(self):
    self._cache.commit(self._progress, self._progress)
    self._cache.clear()

  def installPkg(self, *args):
    for name in args:
      pkg = self._cache[name]
      if pkg.is_installed:
        print(f"{name} is already installed")
      else:
        print(f"Install {name}")
        pkg.mark_install()

  def installDebPackage(self, name):
    apt.debfile.DebPackage(name, self._cache).install()

  def deleteInstalledPkg(self, *args):
    for pkg in self._cache:
      if pkg.is_installed:
        for name in args:
          if pkg.name.startswith(name):
            #print(f"Delete {pkg.name}")
            pkg.mark_delete()

def _setup_nvidia_gl():
  # Install TESLA DRIVER FOR LINUX X64.
  # Kernel module in this driver is already loaded and cannot be neither removed nor updated.
  # (nvidia, nvidia_uvm, nvidia_drm. See dmesg)
  # Version number of nvidia driver for Xorg must match version number of these kernel module.
  # But existing nvidia driver for Xorg might not match.
  # So overwrite them with the nvidia driver that is same version to loaded kernel module.
  ret = subprocess.run(
                  ["nvidia-smi", "--query-gpu=driver_version", "--format=csv,noheader"],
                  stdout = subprocess.PIPE,
                  check = True,
                  universal_newlines = True)
  nvidia_version = ret.stdout.strip()
  nvidia_url = "https://us.download.nvidia.com/tesla/{0}/NVIDIA-Linux-x86_64-{0}.run".format(nvidia_version)
  _download(nvidia_url, "nvidia.run")
  pathlib.Path("nvidia.run").chmod(stat.S_IXUSR)
  subprocess.run(["./nvidia.run", "--no-kernel-module", "--ui=none"], input = "1\n", check = True, universal_newlines = True)

  #https://virtualgl.org/Documentation/HeadlessNV
  subprocess.run(["nvidia-xconfig",
                  "-a",
                  "--allow-empty-initial-configuration",
                  "--virtual=1920x1200",
                  "--busid", "PCI:0:4:0"],
                 check = True
                )

  with open("/etc/X11/xorg.conf", "r") as f:
    conf = f.read()
    conf = re.sub('(Section "Device".*?)(EndSection)',
                  '\\1    MatchSeat      "seat-1"\n\\2',
                  conf,
                  1,
                  re.DOTALL)
  #  conf = conf + """
  #Section "Files"
  #    ModulePath "/usr/lib/xorg/modules"
  #    ModulePath "/usr/lib/x86_64-linux-gnu/nvidia-418/xorg/"
  #EndSection
  #"""

  with open("/etc/X11/xorg.conf", "w") as f:
    f.write(conf)

  #!service lightdm stop
  subprocess.run(["/opt/VirtualGL/bin/vglserver_config", "-config", "+s", "+f"], check = True)
  #user_name = "colab"
  #!usermod -a -G vglusers $user_name
  #!service lightdm start

  # Run Xorg server
  # VirtualGL and OpenGL application require Xorg running with nvidia driver to get Hardware 3D Acceleration.
  #
  # Without "-seat seat-1" option, Xorg try to open /dev/tty0 but it doesn't exists.
  # You can create /dev/tty0 with "mknod /dev/tty0 c 4 0" but you will get permision denied error.
  subprocess.Popen(["Xorg", "-seat", "seat-1", "-allowMouseOpenFail", "-novtswitch", "-nolisten", "tcp"])

def virtual():

  virtualGL_ver = "2.6.4"
  virtualGL_url = "https://github.com/demotomohiro/turbovnc/releases/download/2.2.5/virtualgl_{0}_amd64.deb".format(virtualGL_ver)
  _download(virtualGL_url, "virtualgl.deb")

  my_apt = _MyApt()
  my_apt.installDebPackage("virtualgl.deb")

  my_apt.commit()
  my_apt.close()

  _setup_nvidia_gl()

virtual()
print("Done")

Label(value='')

FloatProgress(value=0.0, layout=Layout(border='1px solid #118800'), max=1.0)

Done


## **Method 0 - Fake Display**

In [ ]:
#@title **Creating a display**
#@markdown The simulation environment will be able to work on gpu with this display/screen and OpenGL.

!sudo apt-get install lightdm
!sudo service lightdm start
!sudo apt-get install screen

clear_output()

**Open the terminal in Colab.**

![Screenshot_2021-04-14_11-34-16](https://user-images.githubusercontent.com/54184905/114680369-c5014180-9d15-11eb-8673-82b03fabcfdc.png)

**Run the "sudo screen" command and skip the warning messages with the space key, so our screen will be launched.**


![Screenshot_2021-04-14_11-39-27](https://user-images.githubusercontent.com/54184905/114680809-38a34e80-9d16-11eb-8401-0e2c3c98c68a.png)

**Switch to RLTurkey user with the "su RLTurkey" command, then access the environment where the simulation is installed with the "cd /content/LinuxNoEditor" command and run the simulation with the "vglrun ./BridgeScene.sh" command.**

![Screenshot_2021-04-14_11-44-46](https://user-images.githubusercontent.com/54184905/114681665-0cd49880-9d17-11eb-8477-f6080e6855b3.png)

![Screenshot_2021-04-14_11-50-00](https://user-images.githubusercontent.com/54184905/114682285-9c7a4700-9d17-11eb-8dbf-cc90e0874369.png)

![Screenshot_2021-04-14_11-51-09](https://user-images.githubusercontent.com/54184905/114682449-c6336e00-9d17-11eb-8ac2-8d26ddc609eb.png)



## **Method 1 - Remote desktop with OpenGL**

In [ ]:
#@title **Remote access to the machine**
#@markdown Creating a new Dısplay and accessing this screen remotely with google chrome ,It takes 4-5 minutes for installation.

#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication

LinuxCommand = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AY0e-g5-t5jHiZFj0zlYDOo55Lc-kvLdNOwalyQy599G4dz8lpUQj8Q0VEmL2Wl4gqWMGw\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}

def CRD():
    with open('install.sh', 'w') as script:
        script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Loading Installer $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Installer Loaded $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Installing Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 desktop-base
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'  
    sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Installed $endc$enda\n" >&2 ||
printf "\r$r$b    Error Occured $endc$enda\n" >&2
sleep 3

printf "$g$b    Installing other Tools $endc$enda" >&2
if sudo apt install nautilus nano -y &> /dev/null
then
    printf "\r$c$b    Other Tools Installed $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Installation Completed $endc$enda\n\n" >&2""")

    ! chmod +x install.sh
    ! ./install.sh

    # Adding user to CRP group
    ! sudo adduser $username chrome-remote-desktop &> /dev/null

    # Finishing Work
    ! su - $username -c """$LinuxCommand"""

try:
    if username:
        if LinuxCommand == "" :
            print("Please enter authcode from the given link")
        else:
            CRD()
except NameError:
    print("username variable not found")
    print("Create a User First")

clear_output()
print("Finished Succesfully")

Finished Succesfully


**After these actions, click on the google sign on the top right and go to the home page.**

![Screenshot_2021-04-14_11-55-26](https://user-images.githubusercontent.com/54184905/114683969-25de4900-9d19-11eb-82db-ab271755faee.png)

**Here, click on the "Access my computer" button on the top right.**

![Screenshot_2021-04-14_11-55-44](https://user-images.githubusercontent.com/54184905/114684260-6b027b00-9d19-11eb-8a4e-2a93fc1839de.png)

**Finally, click on your computer and enter the password you have set, now you are visually connected to the computer in Google Colab :).**

![Screenshot_2021-04-14_11-58-11](https://user-images.githubusercontent.com/54184905/114684568-b4eb6100-9d19-11eb-9cb8-fa43a815895e.png)


![Screenshot_2021-04-14_12-05-51](https://user-images.githubusercontent.com/54184905/114684653-caf92180-9d19-11eb-93b5-a2751e992b81.png)


In [ ]:
#@title **Xfce terminal setup and settings**
#@markdown With this terminal, we will be able to communicate with the operating system and run our commands.

!apt install xfce4 xfce4-goodies
!wget -P /home/RLTurkey/Desktop/ https://ahmetfurkandemir.s3.amazonaws.com/xfce4-terminal.desktop

clear_output()
print("Done")

Done


You can find out where the DISPLAY is opened with the "echo $DISPLAY" command.

![Screenshot_2021-04-13_13-10-38](https://user-images.githubusercontent.com/54184905/114536436-c10fe980-9c59-11eb-9e04-b3f818dcd472.png)


Run the simulation or other applications with this command (Rearrange the command according to the DISPLAY value.). Command : "DISPLAY=:20.0 vglrun ./BridgeScene.sh"

![Screenshot_2021-04-13_13-07-39](https://user-images.githubusercontent.com/54184905/114536199-8312c580-9c59-11eb-86db-497e03d4c0a1.png)

![ezgif com-video-to-gif](https://user-images.githubusercontent.com/54184905/114538503-22d15300-9c5c-11eb-985e-5f87b25e7c35.gif)

![Screenshot_2021-04-13_13-02-33](https://user-images.githubusercontent.com/54184905/114536790-1b10af00-9c5a-11eb-8786-035b80703d6c.png)
